## DSCC202-402 Data Science at Scale Final Project
### Tracking Tweet sentiment at scale using a pretrained transformer (classifier)
<p>Consider the following illustration of the end to end system that you will be building.  Each student should do their own work.  The project will demonstrate your understanding of Spark Streaming, the medalion data architecture using Delta Lake, Spark Inference at Scale using an MLflow packaged model as well as Exploritory Data Analysis and System Tracking and Monitoring.</p>
<br><br>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/pipeline.drawio.png">

<p>
You will be pulling an updated copy of the course GitHub repositiory: <a href="https://github.com/lpalum/dscc202-402-spring2025">The Repo</a>.  

Once you have updated your fork of the repository you should see the following template project that is resident in the final_project directory.
</p>

<img src="https://data-science-at-scale.s3.amazonaws.com/images/notebooks.drawio.png">

<p>
You can then pull your project into the Databrick Workspace using the <a href="https://github.com/apps/databricks">Databricks App on Github</a> or by cloning the repo to your laptop and then uploading the final_project directory and its contents to your workspace using file imports.  Your choice.

<p>
Work your way through this notebook which will give you the steps required to submit a complete and compliant project.  The following illustration and associated data dictionary specifies the transformations and data that you are to generate for each step in the medallion pipeline.
</p>
<br><br>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/dataframes.drawio.png">

#### Bronze Data - raw ingest
- date - string in the source json
- user - string in the source json
- text - tweet string in the source json
- sentiment - the given sentiment of the text as determined by an unknown model that is provided in the source json
- source_file - the path of the source json file the this row of data was read from
- processing_time - a timestamp of when you read this row from the source json

#### Silver Data - Bronze Preprocessing
- timestamp - convert date string in the bronze data to a timestamp
- mention - every @username mentioned in the text string in the bronze data gets a row in this silver data table.
- cleaned_text - the bronze text data with the mentions (@username) removed.
- sentiment - the given sentiment that was associated with the text in the bronze table.

#### Gold Data - Silver Table Inference
- timestamp - the timestamp from the silver data table rows
- mention - the mention from the silver data table rows
- cleaned_text - the cleaned_text from the silver data table rows
- sentiment - the given sentiment from the silver data table rows
- predicted_score - score out of 100 from the Hugging Face Sentiment Transformer
- predicted_sentiment - string representation of the sentiment
- sentiment_id - 0 for negative and 1 for postive associated with the given sentiment
- predicted_sentiment_id - 0 for negative and 1 for positive assocaited with the Hugging Face Sentiment Transformer

#### Application Data - Gold Table Aggregation
- min_timestamp - the oldest timestamp on a given mention (@username)
- max_timestamp - the newest timestamp on a given mention (@username)
- mention - the user (@username) that this row pertains to.
- negative - total negative tweets directed at this mention (@username)
- neutral - total neutral tweets directed at this mention (@username)
- positive - total positive tweets directed at this mention (@username)

When you are designing your approach, one of the main decisions that you will need to make is how you are going to orchestrate the streaming data processing in your pipeline.  There are several valid approaches to triggering your steams and how you will gate the execution of your pipeline.  Think through how you want to proceed and ask questions if you need guidance. The following references may be helpful:
- [Spark Structured Streaming Programming Guide](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html)
- [Databricks Autoloader - Cloudfiles](https://docs.databricks.com/en/ingestion/auto-loader/index.html)
- [In class examples - Spark Structured Streaming Performance](https://dbc-f85bdc5b-07db.cloud.databricks.com/editor/notebooks/2638424645880316?o=1093580174577663)

### Be sure your project runs end to end when *Run all* is executued on this notebook! (7 points)

### This project is worth 25% of your final grade.
- DSCC-202 Students have 55 possible points on this project (see points above and the instructions below)
- DSCC-402 Students have 60 possible points on this project (one extra section to complete)

In [0]:
%run ./includes/includes

In [0]:
"""
Adding a widget to the notebook to control the clearing of a previous run.
or stopping the active streams using routines defined in the utilities notebook
"""
dbutils.widgets.removeAll()

dbutils.widgets.dropdown("clear_previous_run", "No", ["No","Yes"])
if (getArgument("clear_previous_run") == "Yes"):
    clear_previous_run()
    print("Cleared all previous data.")

dbutils.widgets.dropdown("stop_streams", "No", ["No","Yes"])
if (getArgument("stop_streams") == "Yes"):
    stop_all_streams()
    print("Stopped all active streams.")

dbutils.widgets.dropdown("optimize_tables", "No", ["No","Yes"])
if (getArgument("optimize_tables") == "Yes"):
    # Suck up those small files that we have been appending.
    # Optimize the tables
    optimize_table(BRONZE_DELTA)
    optimize_table(SILVER_DELTA)
    optimize_table(GOLD_DELTA)
    print("Optimized all of the Delta Tables")

## 1.0 Import your libraries here (2 points)
- Are your shuffle partitions consistent with your cluster and your workload?
- Do you have the necessary libraries to perform the required operations in the pipeline/application?

In [0]:
# ENTER YOUR CODE HERE

# Parallelism
spark.conf.set("spark.sql.shuffle.partitions", "8")
spark.conf.set("spark.default.parallelism", "8")
spark.conf.set("spark.sql.adaptive.enabled", "true")
spark.conf.set("spark.streaming.backpressure.enabled", "true")

stop_all_streams()
clear_previous_run()

## 2.0 Define and execute utility functions (3 points)
- Read the source file directory listing
- Count the source files (how many are there?)
- print the contents of one of the files

In [0]:
# ENTER YOUR CODE HERE
files = dbutils.fs.ls(TWEET_SOURCE_PATH)
first_file = files[0].path
print(f"There are {len(files)} files")
dbutils.fs.head(first_file)

## 3.0 Transform the Raw Data to Bronze Data using a stream  (8 points)
- define the schema for the raw data
- setup a read stream using cloudfiles and the source data format
- setup a write stream using delta lake to append to the bronze delta table
- enforce schema
- allow a new schema to be merged into the bronze delta table
- Use the defined BRONZE_CHECKPOINT and BRONZE_DELTA paths defined in the includes
- name your raw to bronze stream as bronze_stream
- transform the raw data to the bronze data using the data definition at the top of the notebook

In [0]:
# TODO - Set maxFilesPerTrigger
# TODO - adjust processing time to be higher

from pyspark.sql.functions import current_timestamp, input_file_name

raw_schema = """
date string,
user string,
text string,
sentiment string
"""

spark.sql(f"""
CREATE TABLE IF NOT EXISTS BRONZE_TABLE (
    date string,
    user string,
    text string,
    sentiment string,
    source_file string,
    processing_time timestamp
)
USING DELTA
LOCATION '{BRONZE_DELTA}'
""")

# Want all the streams to be treated the same
spark.sparkContext.setLocalProperty("spark.scheduler.pool", "bronze")

bronze_stream = (spark
      .readStream
      .format("cloudFiles")
      .option("cloudFiles.format", "json")
      .option("maxFilesPerTrigger", 16)
      .schema(raw_schema)
      .load(TWEET_SOURCE_PATH)
      .coalesce(8)
      .withColumn("source_file", input_file_name())
      .withColumn("processing_time", current_timestamp())
      .writeStream
      .format("delta")
      .outputMode("append")
      .queryName("bronze_stream")
      .trigger(processingTime="2 seconds")
      .option("mergeSchema", "true")
      .option("checkpointLocation", BRONZE_CHECKPOINT)
      .start(BRONZE_DELTA)
)

## 4.0 Transform the Bronze Data to Silver Data using a stream (5 points)
- setup a read stream on your bronze delta table
- setup a write stream to append to the silver delta table
- Use the defined SILVER_CHECKPOINT and SILVER_DELTA paths in the includes
- name your bronze to silver stream as silver_stream
- transform the bronze data to the silver data using the data definition at the top of the notebook

In [0]:
# ENTER YOUR CODE HERE
from pyspark.sql.functions import to_timestamp, regexp_extract_all, array_distinct, explode, col, regexp_replace, lit

spark.sql(f"""
CREATE TABLE IF NOT EXISTS SILVER_TABLE (
    timestamp timestamp not null,
    mention string not null,
    cleaned_text string not null,
    Sentiment string not null
)
USING DELTA
LOCATION '{SILVER_DELTA}'
""")

# Setting for time format
spark.conf.set("spark.sql.legacy.timeParserPolicy","LEGACY")

# Want all the streams to be treated the same
spark.sparkContext.setLocalProperty("spark.scheduler.pool", "silver")

wait_for_delta_table(BRONZE_DELTA)

silver_stream = (spark
      .readStream
      .format("delta")
      .load(BRONZE_DELTA)
      .withColumn("timestamp", to_timestamp(col("date"), "E MMM dd HH:mm:ss z yyyy"))
      .withColumn("mention", 
                  explode(
                      regexp_extract_all(col("text"), lit(r'@\w+'), 0)))
      .withColumn("cleaned_text", regexp_replace(col("text"), r"@\w+", ""))
      .select("timestamp", "mention", "cleaned_text", col("sentiment").alias("Sentiment"))
      .writeStream
      .format("delta")
      .outputMode("append")
      .queryName("silver_stream")
      .trigger(processingTime="4 seconds")
      .option("checkpointLocation", SILVER_CHECKPOINT)
      .start(SILVER_DELTA)
)

## 5.0 Transform the Silver Data to Gold Data using a stream (7 points)
- setup a read stream on your silver delta table
- setup a write stream to append to the gold delta table
- Use the defined GOLD_CHECKPOINT and GOLD_DELTA paths defines in the includes
- name your silver to gold stream as gold_stream
- transform the silver data to the gold data using the data definition at the top of the notebook
- Load the pretrained transformer sentiment classifier from the MODEL_NAME at the production level from the MLflow registry
- Use a spark UDF to parallelize the inference across your silver data

In [0]:
# ENTER YOUR CODE HERE
import mlflow
from pyspark.sql.functions import col, when, lit

# Model info
model_uri = f"models:/{MODEL_NAME}/Production"
predict = mlflow.pyfunc.spark_udf(spark, model_uri)

# Want all the streams to be treated the same
spark.sparkContext.setLocalProperty("spark.scheduler.pool", "gold")

# TODO - Need to decrease offset read in at a time

wait_for_delta_table(SILVER_DELTA)

gold_stream = (spark
      .readStream
      .format("delta")
      .option("maxOffsetsPerTrigger", 550)
      .load(SILVER_DELTA)
      .withColumn("prediction", predict(col("cleaned_text")))
      .withColumn("predicted_score", col("prediction").score)  
      .withColumn("predicted_sentiment", col("prediction").label)
      .withColumn("sentiment_id", when(col("sentiment") == "positive", lit(1)).otherwise(lit(0)))
      .withColumn("predicted_sentiment_id", when(col("predicted_sentiment") == "NEG", lit(0)).otherwise(lit(1)))
      .drop("prediction")
      .writeStream
      .format("delta")
      .outputMode("append")
      .queryName("gold_stream")
      .trigger(processingTime="20 seconds")
      .option("checkpointLocation", GOLD_CHECKPOINT)
      .start(GOLD_DELTA)
)

## 6.0 Monitor your Streams (5 points)
- Setup a loop that runs at least every 10 seconds
- Print a timestamp of the monitoring query along with the list of streams, rows processed on each, and the processing time on each
- Run the loop until all of the data is processed (0 rows read on each active stream)
- Plot a line graph that shows the data processed by each stream over time
- Plot a line graph that shows the average processing time on each stream over time

In [0]:
# ENTER YOUR CODE HERE
from datetime import datetime
import matplotlib.pyplot as plt

# Print stream stats
for i in range(0, 600):
    time.sleep(10)
    
    stats = get_streaming_stats()

    # Break if not streams active
    is_active = False
    for _, row in stats.groupby("query").last().reset_index().iterrows():
        if row["input_rows"] > 0:
            is_active = True
            break
        
    if not is_active:
        break

    # Display latest stats and query info
    latest_stats = stats.sort_values("elapsed_time").groupby("query").last().reset_index()
    print(f"\nStatus at {datetime.now().strftime('%H:%M:%S')} (Elapsed: {i*10}s):")

    for _, row in latest_stats.iterrows():
        print(f"  {row['query']}: {row['input_rows']} rows, {row['processing_time']}ms processing time")

# Print final results
df = get_streaming_stats()
if not df.empty:
    # Create a figure with two subplots
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), sharex=True)
    
    # Plot processing time
    for query in df["query"].unique():
        subset = df[df["query"] == query]
        ax1.plot(subset["elapsed_time"], subset["processing_time"], marker='o', linestyle='-', label=query)
    
    ax1.set_ylabel("Processing Time (ms)")
    ax1.set_title(f"Spark Streaming Processing Time")
    ax1.legend()
    ax1.grid(True)
    
    # Plot input rows
    for query in df["query"].unique():
        subset = df[df["query"] == query]
        ax2.plot(subset["elapsed_time"], subset["input_rows"], marker='o', linestyle='-', label=query)
    
    ax2.set_xlabel("Elapsed Time (seconds)")
    ax2.set_ylabel("Input Rows")
    ax2.set_title(f"Spark Streaming Input Rows")
    ax2.legend()
    ax2.grid(True)
    
    plt.tight_layout()
    plt.show()
    
    # Generate statistics summary
    print("\nStreaming Performance Summary:")
    summary = df.groupby("query").agg({
        "processing_time": ["mean", "max", "min", "std"],
        "input_rows": ["sum", "mean", "max"]
    }).reset_index()
    display(summary)
    

## 7.0 Bronze Data Exploratory Data Analysis (5 points)
- How many tweets are captured in your Bronze Table?
- Are there any columns that contain Nan or Null values?  If so how many and what will you do in your silver transforms to address this?
- Count the number of tweets by each unique user handle and sort the data by descending count.
- How many tweets have at least one mention (@) how many tweet have no mentions (@)
- Plot a bar chart that shows the top 20 tweeters (users)


In [0]:
# ENTER YOUR CODE HERE

from pyspark.sql.functions import isnotnull, size
import matplotlib.pyplot as plt

# # Count number of tweets
bronze_df = (spark
    .read
    .format("delta")
    .option("inferSchema", raw_schema)
    .load(BRONZE_DELTA)
    .cache())

print(f"Bronze Table has {bronze_df.count()} records")

# # Look for null values
contains_null = not bronze_df.filter(
    isnotnull("date") & 
    isnotnull("user") & 
    isnotnull("text") & 
    isnotnull("sentiment")
).count() == bronze_df.count()

print(f"Bronze Table has null records: {contains_null}")

# # Get count by user handle 
tweets_by_unique_handle = (bronze_df
    .groupBy("user")
    .count()
    .sort("count", ascending=False))

display(tweets_by_unique_handle)

# # Number of tweets with 1 or more mentions
mentions = (bronze_df
    .withColumn("mentions", regexp_extract_all(col("text"), lit(r'@\w+'), 0))
    .filter(size("mentions") > 0)
    .count())

print(f"There are {mentions} tweets with 1 or more mentions")
print(f"There are {bronze_df.count() - mentions} tweets with 0 mentions")

# Plot a barchart with the top 20 tweeters
fig, ax = plt.subplots()
pandas_df = tweets_by_unique_handle.limit(20).toPandas()

ax.bar(pandas_df["user"], pandas_df["count"])
ax.set_title("Top 20 Tweeters")
ax.set_ylabel("Number of Tweets")
ax.set_xlabel("Handles")
ax.set_xticklabels(pandas_df["user"], rotation=-90)

plt.tight_layout()
plt.show()


## 8.0 Capture the accuracy metrics from the gold table in MLflow  (4 points)
Store the following in an MLflow experiment run:
- Store the precision, recall, and F1-score as MLflow metrics
- Store an image of the confusion matrix as an MLflow artifact
- Store the model name and the MLflow version that was used as an MLflow parameters
- Store the version of the Delta Table (input-silver) as an MLflow parameter

In [0]:
# ENTER YOUR CODE HERE
from mlflow.tracking import MlflowClient
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

client = MlflowClient()
model = client.get_latest_versions(MODEL_NAME, stages=['Production'])[0]
data_version = spark.sql(f"DESCRIBE HISTORY '{SILVER_DELTA}' LIMIT 1").first()['version']

results_df = (spark
           .read
           .format("delta")
           .load(GOLD_DELTA)
           .select("sentiment_id", "predicted_sentiment_id")
           .toPandas())

y = results_df['sentiment_id']
y_hat = results_df['predicted_sentiment_id']

with mlflow.start_run(run_name="accuracy-metrics"):
    # Model name and version
    mlflow.log_param("model_name", MODEL_NAME)
    mlflow.log_param("model_version", model.version)
    mlflow.log_param("input-silver", data_version)

    # Log precision, recall, F1-score metrics
    report = classification_report(y, y_hat, output_dict=True)
    mlflow.log_metric('precision_neg', report['0']['precision'])
    mlflow.log_metric('precision_pos', report['1']['precision'])
    mlflow.log_metric('recall_neg', report['0']['recall'])
    mlflow.log_metric('recall_pos', report['0']['recall'])
    mlflow.log_metric('f1_neg', report['0']['f1-score'])
    mlflow.log_metric('f1_pos', report['1']['f1-score'])

    # Store the confusion matrix as an artifact
    cm = confusion_matrix(y, y_hat)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["NEG", "POS"])

    disp.plot()
    fig = plt.gcf()
    mlflow.log_figure(fig, "confusion_matrix.png")

## 9.0 Application Data Processing and Visualization (6 points)
- How many mentions are there in the gold data total?
- Count the number of neutral, positive and negative tweets for each mention in new columns
- Capture the total for each mention in a new column
- Sort the mention count totals in descending order
- Plot a bar chart of the top 20 mentions with positive sentiment (the people who are in favor)
- Plot a bar chart of the top 20 mentions with negative sentiment (the people who are the vilians)

*note: A mention is a specific twitter user that has been "mentioned" in a tweet with an @user reference.

In [0]:
# ENTER YOUR CODE HERE

from pyspark.sql.functions import count

gold_df = (spark
           .read
           .format("delta")
           .load(GOLD_DELTA))

# How many mentions are in the gold data?
print(f"There are {gold_df.count()} mentions")

# Count the number of neutral positive and negative tweets for each mention
# Sort mention count totals in descending order
sentiment_count = (gold_df
    .groupBy("mention")
    .agg(
        count(when(col("predicted_sentiment") == "POS", True)).alias("positive"),
        count(when(col("predicted_sentiment") == "NEU", True)).alias("neutral"),
        count(when(col("predicted_sentiment") == "NEG", True)).alias("negative")
     )
    .withColumn("total_mentions", col("positive") + col("neutral") + col("negative"))
    .orderBy(col("total_mentions").desc())
    .cache())

display(sentiment_count)

t20_pos = (sentiment_count
    .select("mention", "positive")
    .orderBy(col("positive").desc())
    .limit(20)
    .toPandas())

t20_neg = (sentiment_count
    .select("mention", "negative")
    .orderBy(col("negative").desc())
    .limit(20)
    .toPandas())

# Plot the data
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(9, 11))

ax1.bar(t20_pos["mention"], t20_pos["positive"])
ax1.set_title("Top 20 Handles by Positive Sentiment")
ax1.set_ylabel("Number of Postive Mentions")
ax1.set_xlabel("Handle")
ax1.set_xticklabels(t20_pos["mention"], rotation=-90)

ax2.bar(t20_neg["mention"], t20_neg["negative"])
ax2.set_title("Top 20 Handles by Negative Sentiment")
ax2.set_ylabel("Number of Negative Mentions")
ax2.set_xlabel("Handle")
ax2.set_xticklabels(t20_neg["mention"], rotation=-90)

plt.tight_layout()
plt.show()


## 10.0 Clean up and completion of your pipeline (3 points)
- using the utilities what streams are running? If any.
- Stop all active streams
- print out the elapsed time of your notebook. Note: In the includes there is a variable START_TIME that captures the starting time of the notebook.

In [0]:
# ENTER YOUR CODE HERE

# What streams are running
print("Streams running:")
for stream in spark.streams.active:
    print(stream.name)

# Stop all streams and clear all the tables
stop_all_streams()
clear_previous_run()

# Elapsed time
print(f"{time.time() - START_TIME} seconds have elapsed")

## 11.0 How Optimized is your Spark Application (Grad Students Only) (5 points)
Graduate students (registered for the DSCC-402 section of the course) are required to do this section.  This is a written analysis using the Spark UI (link to screen shots) that support your analysis of your pipelines execution and what is driving its performance.
Recall that Spark Optimization has 5 significant dimensions of considertation:
- Spill: write to executor disk due to lack of memory
- Skew: imbalance in partition size
- Shuffle: network io moving data between executors (wide transforms)
- Storage: inefficiency due to disk storage format (small files, location)
- Serialization: distribution of code segments across the cluster

Comment on each of the dimentions of performance and how your impelementation is or is not being affected.  Use specific information in the Spark UI to support your description.  

Note: you can take sreenshots of the Spark UI from your project runs in databricks and then link to those pictures by storing them as a publicly accessible file on your cloud drive (google, one drive, etc.)

References:
- [Spark UI Reference Reference](https://spark.apache.org/docs/latest/web-ui.html#web-ui)
- [Spark UI Simulator](https://www.databricks.training/spark-ui-simulator/index.html)

### ENTER YOUR MARKDOWN HERE